In [1]:
import pandas as pd
from sql import get_data
import warnings
warnings.filterwarnings("ignore")
import psycopg2 # Needed to get database errors when uploading dataframe
import sqlalchemy
from geopy.geocoders import Nominatim

In [2]:
translator = Translator()

In [8]:
clean_jobs = get_data('SELECT * FROM capstone.jobs_translated')

## Translate Job title and Job description to english

In [12]:
job_translate = pd.Series([])

for row in range(len(clean_jobs)):
    value = translator.translate(clean_jobs.loc[row, 'job_title'])
    job_translate[row] = value.text

clean_jobs.insert(5, "job_title_en2", job_translate) 


/var/folders/10/lgjgf_c13nj8fhnrmny3d2br0000gn/T/ipykernel_1689/2002001171.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  job_translate = pd.Series([])


In [63]:
job_text_translate = pd.Series([])

for row in range(len(clean_jobs)):
    value = translator.translate(clean_jobs.loc[row, 'full_text'])
    job_text_translate[row] = value.text

clean_jobs.insert(7, "job_description", job_text_translate) 
    #clean_jobs

## Delete unnecessary columns

In [102]:
clean_jobs = clean_jobs.drop(columns='world_region',axis=1)

## Create new column with the US states and write 'USA' in the country column

In [26]:
us_list = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 
            'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 
            'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 
            'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 
            'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

us_states = pd.Series([])

for row in range(len(clean_jobs)):

    if clean_jobs['country'][row] in us_list:
        us_states[row] = clean_jobs['country'][row]
        clean_jobs['country'][row] = 'USA'
    

clean_jobs.insert(6, "us_state", us_states) 

## Create dict with region categories

In [17]:
region_category = {
    'Europe' : ['Ukraine', 'France', 'Spain', 'Sweden', 'Norway', 'Germany', 'Finland', 'Poland', 'Italy', 'United Kingdom', 
                'Romania', 'Belarus', 'Kazakhstan', 'Greece', 'Bulgaria', 'Iceland', 'Hungary', 'Portugal', 'Serbia', 'Austria', 
                'Czech Republic', 'Ireland', 'Lithuania', 'Latvia', 'Croatia', 'Bosnia and Herzegovina', 'Slovakia', 'Estonia', 
                'Denmark', 'Switzerland', 'Netherlands', 'Moldova', 'Belgium', 'Albania', 'North Macedonia', 'Slovenia', 
                'Montenegro', 'Kosovo', 'Azerbaijan', 'Luxembourg', 'Andorra', 'Malta', 'Liechtenstein', 'San Marino', 
                'Monaco', 'Vatican City', 'Cyprus',  'Czechia'],

    'Africa' : ['Nigeria', 'Ethiopia', 'Egypt', 'DR Congo', 'Tanzania', 'South Africa', 'Kenya', 'Uganda', 'Algeria', 
                'Sudan', 'Morocco', 'Angola', 'Mozambique', 'Ghana', 'Madagascar', 'Cameroon', 'Niger', 'Burkina Faso', 'Mali', 
                'Malawi', 'Zambia', 'Senegal', 'Chad', 'Somalia', 'Zimbabwe', 'Guinea', 'Rwanda', 'Benin', 'Burundi', 'Tunisia', 
                'South Sudan', 'Togo', 'Sierra Leone', 'Libya', 'Congo', 'Liberia', 'Central African Republic', 'Mauritania', 'Eritrea', 
                'Namibia', 'Gambia', 'Botswana', 'Gabon', 'Lesotho', 'Guinea-Bissau', 'Equatorial Guinea', 'Mauritius', 'Eswatini', 'Djibouti'],

'Middle East' : ['Turkey', 'Bahrain', 'Kuwait', 'Oman', 'Qatar', 'Saudi Arabia', 'United Arab Emirates', 'Yemen', 'Abkhazia', 'Armenia', 
                'Artsakh', 'Azerbaijan', 'Georgia', 'South Ossetia', 'Iraq', 'Israel', 'Jordan', 'Lebanon', 'Palestine', 'Syria', 'Iran', 
                'Akrotiri and Dhekelia', 'Cyprus'],
                
'North America' : ['USA', 'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 
                'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 
                'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 
                'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 
                'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 
                'West Virginia', 'Wisconsin', 'Wyoming', 'Canada', 'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 
                'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 
                'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'],

'Central America' : ['Mexico', 'Guatemala', 'Cuba', 'Haiti', 'Dominican Republic', 'Honduras', 'Nicaragua', 'El Salvador', 'Costa Rica', 'Panama', 
                    'Jamaica', 'Trinidad and Tobago', 'Belize', 'Bahamas', 'Barbados', 'Saint Lucia', 'Grenada', 'Antigua and Barbuda', 'Dominica', 
                    'Saint Kitts and Nevis'],

'South America' : ['Argentina', 'Bolivia', 'Bouvet Island', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Falkland Islands', 'French Guiana', 
                'Guyana', 'Paraguay', 'Peru', 'South Georgia', 'Suriname', 'Uruguay', 'Venezuela', 'South Sandwich Islands'],

'South Asia' : ['Afghanistan', 'Bangladesh', 'Bhutan', 'India', 'Nepal', 'Maldives', 'Pakistan', 'Sri Lanka'],

'Southeast Asia' : ['Brunei', 'Cambodia', 'East Timor', 'Indonesia', 'Laos', 'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Vietnam'],

'Central Asia' : ['Kazakhstan', 'Kyrgyzstan', 'Tajikistan', 'Turkmenistan', 'Uzbekistan'],

'East Asia' : ['China', 'Hong Kong', 'Macau', 'Japan', 'Mongolia', 'North Korea', 'South Korea', 'Taiwan']

}

In [28]:
clean_jobs.head(25)

,id,job_id,posted_date,company_name,job_title_en,full_text,world_region,state,job_description,city,country,geo_location
0,584,2022-04-28_Bruchsal_Flight Simulator Engineer ...,2022-04-28,Volocopter GmbH,Flight Simulator Engineer (m/f/d),Your mission during this flight:As Flight Simu...,Europe,None,Your mission during this flight:As Flight Simu...,Bruchsal,Germany,"(Bruchsal, Verwaltungsgemeinschaft Bruchsal, L..."
1,585,2022-04-07_Munich_Embedded Communication Elect...,2022-04-07,Volocopter GmbH,Embedded Communication Electronics - VoloConne...,Your future Volocopter crew:Within a highly mo...,Europe,None,Your future Volocopter crew:Within a highly mo...,Munich,Germany,"(München, Bayern, Deutschland, (48.1371079, 11..."
2,586,2022-05-25_Hamburg_Shell Graduate Programme 20...,2022-05-25,Shell,Shell Graduate Programme 2022 (Commercial) - G...,Are you in the final year of your studies or g...,Europe,None,Are you in the final year of your studies or g...,Hamburg,Germany,"(Hamburg, 20095, Deutschland, (53.550341, 10.0..."
3,339,2022-05-16_Greater Marseille Metropolitan Area...,2022-05-16,FERCHAU Aviation Group - France,Project Manager,Descriptif du poste : Chef de projet simulatio...,Europe,None,Job description: Flight simulation project man...,Greater Marseille Metropolitan Area,France,None
4,306,2022-03-22_Paris_Infrastructure Engineer - (Fl...,2022-03-22,SITA,Infrastructure Engineer - (Flex/Hybrid Options),As owner of one of the largest secure private ...,Europe,None,As owner of one of the largest secure private ...,Paris,France,"(Paris, Île-de-France, France métropolitaine, ..."
5,307,2022-05-16_Aix-en-Provence_Développeur back-end,2022-05-16,Cap4 Lab,Back end developer,"Avec Cap4 Lab, vous rejoignez une entreprise i...",Europe,None,"With Cap4 Lab, you join a growing internationa...",Aix-en-Provence,France,"(Aix-en-Provence, Bouches-du-Rhône, Provence-A..."
6,308,"2022-05-11_Us_Director of Health, Safety & Env...",2022-05-11,Modern Aviation,"Director of Health, Safety & Environment",Primary Purpose of the PositionCreates and imp...,Europe,None,Primary Purpose of the PositionCreates and imp...,Us,France,"(United States, (39.7837304, -100.445882))"
7,309,2022-05-01_Bouguenais_Ingenieur QSE Alternant H/F,2022-05-01,IRT Jules Verne,Ingenieur QSE Alternant H/F,LInstitut de Recherche Technologique Jules Ver...,Europe,None,The Jules Verne Technological Research Institu...,Bouguenais,France,"(Bouguenais, Nantes, Loire-Atlantique, Pays de..."
8,312,2022-04-23_Cugnaux_Ingénieur Développement Str...,2022-04-23,AURA-AERO,Structural Development Engineer M/F,"Descriptif de l’entreprise ✈AURA AERO, constru...",Europe,None,"Description of the company ✈AURA AERO, a Frenc...",Cugnaux,France,"(Cugnaux, Toulouse, Haute-Garonne, Occitanie, ..."
9,317,2022-05-18_Seine-et-Marne_Ingénieur Assurance ...,2022-05-18,Modis,Aerospace Supplier Quality Assurance Engineer ...,Votre missionModis Tech Talent Services* est l...,Europe,None,Your mission Modis Tech Talent Services* is th...,Seine-et-Marne,France,"(Seine-et-Marne, Île-de-France, France métropo..."


In [10]:
clean_jobs.rename(columns = {'job_title_en2':'job_title_en'}, inplace = True)

## Iterate over region dict to create new column with key=value

In [26]:
world_regions = pd.Series([])

for row in range(len(clean_jobs)):

    for key in region_category:

        for value in region_category[key]:
            
            if clean_jobs['country'][row] == value:
                world_regions[row] = key
   
clean_jobs.insert(6, "world_region", world_regions)

## Creating state names for column state

In [12]:
from geopy.geocoders import Nominatim
 
geolocator = Nominatim(user_agent = "geoapiExercises")

geo_locations = pd.Series([])

for row in range(len(clean_jobs)):
    loc = geolocator.geocode(clean_jobs.loc[row, 'city'])
    geo_locations[row] = loc
        
clean_jobs.insert(10, "geo_location", geo_locations)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Greater+Madrid+Metropolitan+Area&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

# Upload cleaned and revised DataFrame

In [96]:
table_name = 'jobs_translated'
schema = 'capstone'
from sql import engine

if engine!=None:
    try:
        clean_jobs.to_sql(name=table_name,            # Name of SQL table
                        con=engine,                     # Engine or connection
                        if_exists='replace',            # Drop the table before inserting new values 
                        schema='capstone',              # Use schmea that was defined earlier
                        index=False,                    # Write DataFrame index as a column
                        chunksize=5000,                 # Specify the number of rows in each batch to be written at a time
                        method='multi')                 # Pass multiple values in a single INSERT clause
        print(f"The table '{table_name}' was uploaded successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The table 'jobs_translated' was uploaded successfully.
